In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/My Drive')

In [ ]:
import os
# 查看當前工作目錄
retval = os.getcwd()
print("當前工作目錄為 %s" % retval)

當前工作目錄為 /content/drive/My Drive


In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pprint
from google.colab.patches import cv2_imshow
import scipy.ndimage
from scipy.optimize import curve_fit
import csv
from mpl_toolkits.mplot3d import Axes3D
import math
from scipy.signal import find_peaks
import argparse
from operator import sub
from math import atan, pi
from math import sqrt
import random
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
def calc_abc_from_line_2d(x0, y0, x1, y1):
    a = y0-y1
    b = x1-x0
    c = x0*y1-x1*y0
    return a, b, c


def get_line_cross_point(line1, line2):
    a0, b0, c0 = calc_abc_from_line_2d(*line1)
    a1, b1, c1 = calc_abc_from_line_2d(*line2)
    D = a0*b1-a1*b0
    if D==0:
        return None
    x = (b0*c1-b1*c0)/D
    y = (a1*c0-a0*c1)/D
    return x, y

In [ ]:
data_path = '/content/drive/MyDrive/Track_img/train'

for file in os.listdir(data_path):
  filename = os.path.join(data_path,str(file),"0.jpg")
  print(filename)

/content/drive/MyDrive/Track_img/train/00280/0.jpg
/content/drive/MyDrive/Track_img/train/00285/0.jpg
/content/drive/MyDrive/Track_img/train/00291/0.jpg
/content/drive/MyDrive/Track_img/train/00293/0.jpg
/content/drive/MyDrive/Track_img/train/00271/0.jpg
/content/drive/MyDrive/Track_img/train/00264/0.jpg
/content/drive/MyDrive/Track_img/train/00267/0.jpg
/content/drive/MyDrive/Track_img/train/00258/0.jpg
/content/drive/MyDrive/Track_img/train/00270/0.jpg
/content/drive/MyDrive/Track_img/train/00287/0.jpg
/content/drive/MyDrive/Track_img/train/00276/0.jpg
/content/drive/MyDrive/Track_img/train/00259/0.jpg
/content/drive/MyDrive/Track_img/train/00281/0.jpg
/content/drive/MyDrive/Track_img/train/00278/0.jpg
/content/drive/MyDrive/Track_img/train/00268/0.jpg
/content/drive/MyDrive/Track_img/train/00261/0.jpg
/content/drive/MyDrive/Track_img/train/00283/0.jpg
/content/drive/MyDrive/Track_img/train/00266/0.jpg
/content/drive/MyDrive/Track_img/train/00289/0.jpg
/content/drive/MyDrive/Track_im

In [ ]:
data_path = '/content/drive/MyDrive/part2_test_image/train'

for file in os.listdir(data_path):
  filename = os.path.join(data_path,str(file),"0.jpg")
  #filename = '/content/drive/MyDrive/Track_img/train/00002/0.jpg'

  img = cv2.imread(filename)
  hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

  def adjust_gamma(image, gamma=1.0):
      # build a lookup table mapping the pixel values [0, 255] to their adjusted gamma values
      invGamma = 1.0 / gamma
      table = np.array([((i / 255.0) ** invGamma) * 255
          for i in np.arange(0, 256)]).astype("uint8")
      return cv2.LUT(image, table) # apply gamma correction using the lookup table

  # generate green space
  green = np.uint8([[[0,255,0]]])
  range1 = [20,255,200]
  range2 = [20,0,0]
  hsv_green = cv2.cvtColor(green, cv2.COLOR_BGR2HSV)

  # detect green region
  iterations = 2
  mask = cv2.inRange(hsv, hsv_green-range1, hsv_green+range2) > 0
  mask = cv2.erode(mask.astype(np.float32), None, iterations = 1)
  mask = cv2.dilate(mask.astype(np.float32), None, iterations = iterations)
  mask = cv2.erode(mask, None, iterations = iterations+2)
  mask = mask > 0
  mask = np.stack((mask,mask,mask), 2)

  # line detect
  kernel_size = 5

  # sharpen
  gray = adjust_gamma(gray, 0.4)
  blur = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0).astype(int)
  sub = gray.astype(int) - blur
  sharped_img = np.clip(gray.astype(int) + sub*2, a_min = 0, a_max = 255).astype('uint8')

  # sharped_gray = cv2.cvtColor(sharped_img, cv2.COLOR_BGR2GRAY)
  low_threshold = 85
  high_threshold = 85*3
  edges = cv2.Canny(sharped_img, low_threshold, high_threshold, apertureSize=3)

  rho = 1  # distance resolution in pixels of the Hough grid
  theta = np.pi / 180  # angular resolution in radians of the Hough grid
  threshold = 30  # minimum number of votes (intersections in Hough grid cell)
  min_line_length = 75  # minimum number of pixels making up a line
  max_line_gap = 5  # maximum gap in pixels between connectable line segments

  # Run Hough on edge detected image
  # Output "lines" is an array containing endpoints of detected line segments
  lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                      min_line_length, max_line_gap)

  m_lines = []
  def interpolate(pt):
      x1,y1,x2,y2 = pt
      num = max(abs(x1-x2), abs(y1-y2))
      np.around(np.linspace(x1, x2+1, num=num+1))
      return np.stack((np.around(np.linspace(x1, x2, num=num+1)), np.around(np.linspace(y1, y2, num=num+1))), axis=1).astype(int)

  # filter out lines
  for line in lines.reshape(-1,4):
      # interpolate
      full_line = interpolate(line)
      check = mask[:, :,0][full_line[:,1], full_line[:,0]]
      if check.sum() >= len(check) * 0.5:
          m_lines.append(line)
  if len(m_lines)<=10:
    continue
  print(len(m_lines), 'Lines')

  def check_white(hsv, lines, set_i):
      line_mask = np.zeros(hsv.shape[:2], 'uint8')
      for line_id in set_i:
          x1,y1,x2,y2 = lines[line_id]
          cv2.line(line_mask,(x1,y1),(x2,y2),(255,255,255),1)
      line_mask = cv2.dilate(line_mask.astype(np.float32), None, iterations=2) > 0

      white_hsv = np.uint8([[[0,0,255]]])
      range1 = [20,255,200]
      range2 = [20,0,0]
      white_region = cv2.inRange(hsv, white_hsv-[0,0,60], white_hsv+[255,60,0]) > 0

      white_line_region = np.logical_and(line_mask, white_region)

      return (white_line_region.sum() / line_mask.sum()) > 0.03

  def find_edge(p1, p2, h, w):
      v12 = p2-p1 + 1e-18
      target = np.array([0 if v12[0] < 0 else w-1, 0 if v12[1] < 0 else h-1])
      steps = (target - p2)/v12
      edge_p2 = (steps.min()*v12+p2).astype(int)

      v21 = -v12
      target = np.array([0 if v21[0] < 0 else w-1, 0 if v21[1] < 0 else h-1])
      steps = (target - p1)/v21
      edge_p1 = (steps.min()*v21+p1).astype(int)
      if edge_p1[1] == 0: # start with top edge
          return edge_p1.astype(int), edge_p2.astype(int)
      elif edge_p1[0] == 0: # start with left edge
          if edge_p2[1] == 0: # end with top edge
              return edge_p2.astype(int), edge_p1.astype(int)
          else:
              return edge_p1.astype(int), edge_p2.astype(int)
      elif edge_p1[1] == h-1: # start with botton edge
          if edge_p2[0] == w-1: # end with right edge
              return edge_p1.astype(int), edge_p2.astype(int)
          else:
              return edge_p2.astype(int), edge_p1.astype(int)
      elif edge_p1[0] == w-1: # start with right edge
          return edge_p2.astype(int), edge_p1.astype(int)

  def find_intersection(l1, l2):
      with np.errstate(divide='ignore', invalid='ignore'):
          # s * (x - px) + py = y
          s1 = (l1[3] - l1[1]) / (l1[2] - l1[0])
          s2 = (l2[3] - l2[1]) / (l2[2] - l2[0])
          s1 = 1e+20 if np.isinf(s1) else s1
          s2 = 1e+20 if np.isinf(s2) else s2
          px1 = l1[0]
          py1 = l1[1]
          px2 = l2[0]
          py2 = l2[1]
          # [s1 * (x - px1) + py1] - [s2 * (x - px2) + py2] = 0
          x = (-py1+py2 + s1*px1 - s2*px2) / (s1-s2)
          x = 1e+6 if np.isnan(x) else x
          y = s1 * (x - px1) + py1 if (x - px1) != 0 else s2 * (x - px2) + py2
          return (x,y)

  def merge_lines(lines):
      num_l = len(lines)
  #     remove = []
      for l1 in range(num_l):
          for l2 in range(l1+1, num_l):
              if (lines[l1] == lines[l2]).all() or (lines[l1] == np.roll(lines[l2], 2)).all():

                  lines[l2] = np.array([0,0,0,0])
      lines = [i for i in lines if not (i==np.array([0,0,0,0])).all()]
      return lines

  def dist(samples):
      xx,xy = np.meshgrid(samples[:,0], samples[:,0])
      yx,yy = np.meshgrid(samples[:,1], samples[:,1])
      ans = np.sqrt((xy-xx)**2+(yx-yy)**2)
      xx,xy = np.meshgrid(samples[:,2], samples[:,2])
      yx,yy = np.meshgrid(samples[:,3], samples[:,3])
      ans = np.maximum(ans, np.sqrt((xy-xx)**2+(yx-yy)**2))

      ans = np.triu(ans)
      ans[ans==0] = ans.max()+1
      return ans

  def lines_clustering(lines:list, threshold, distance=dist):
      def step(data, group):
          if len(group)==1:
              return
          differ_map = distance(np.array(data))
          min_idx = np.unravel_index(differ_map.argmin(), differ_map.shape)
          if differ_map[min_idx] > threshold:
              return
          data[min_idx[0]] = (data[min_idx[0]] + data[min_idx[1]]) /2
          group[min_idx[0]] = group[min_idx[0]].union(group[min_idx[1]])
          data.pop(min_idx[1])
          group.pop(min_idx[1])
          del differ_map
          step(data, group)

      lines0 = lines.copy()
      group0 = [set([i]) for i in range(len(lines0))]
      step(lines0, group0)
      group1 = []
      for set_i in group0:
          slopes = []
          for idx, line_i in enumerate(set_i):
              (x1,y1,x2,y2) = lines[line_i]
              slopes.append([line_i, atan((y2-y1)/(x2-x1) if (x2-x1)!=0 else 1e+6)+pi/2])

          ele = np.array(slopes)[:,1]
          mean = np.mean(ele, axis=0)
          sd   = np.std(ele, axis=0)
          newset = [x for x in slopes if (x[1] >= mean - 2 * sd)]
          newset = [x for x in newset if (x[1] <= mean + 2 * sd)]
          if len(newset) > 1:
              #print('slopes:',slopes)
              #print('newset:',newset)
              newset = [x[0] for x in newset]
              group1.append(set(newset))
      #print(group1)
      return group1

  def filter_set(lines, sets, _h, _w):
      fake_lines_sets = []
      lines = np.array(lines)
      for seti in sets:
          lines_s_in_set = lines[list(seti),:2]
          lines_e_in_set = lines[list(seti),2:]
          s_min = lines_s_in_set.sum(1).argmin()
          s_max = lines_s_in_set.sum(1).argmax()
          e_min = lines_e_in_set.sum(1).argmin()
          e_max = lines_e_in_set.sum(1).argmax()
          p1 = lines_s_in_set[s_min]
          p2 = lines_s_in_set[s_max]
          p3 = lines_e_in_set[e_min]
          p4 = lines_e_in_set[e_max]
          inter = find_intersection(np.concatenate((p1, p3), axis=0), np.concatenate((p2, p4), axis=0))
          if inter[0] < 0 or inter[0] >= _w or inter[1] < 0 or inter[1] >= _h:
              if p1[1] == 0:
                  if p1.sum() > p2.sum():
                      fake_lines_sets.append([np.concatenate((p1, p3), axis=0), np.concatenate((p2, p4), axis=0)])
                  else:
                      fake_lines_sets.append([np.concatenate((p2, p4), axis=0), np.concatenate((p1, p3), axis=0)])
              else:
                  if p1.sum() > p2.sum():
                      fake_lines_sets.append([np.concatenate((p2, p4), axis=0), np.concatenate((p1, p3), axis=0)])
                  else:
                      fake_lines_sets.append([np.concatenate((p1, p3), axis=0), np.concatenate((p2, p4), axis=0)])
          else:

              if p1[1] == 0:
                  if p1.sum() > p2.sum():
                      fake_lines_sets.append([np.concatenate((p1, p4), axis=0), np.concatenate((p2, p3), axis=0)])
                  else:
                      fake_lines_sets.append([np.concatenate((p2, p3), axis=0), np.concatenate((p1, p4), axis=0)])
              else:
                  if p1.sum() > p2.sum():
                      fake_lines_sets.append([np.concatenate((p2, p3), axis=0), np.concatenate((p1, p4), axis=0)])
                  else:
                      fake_lines_sets.append([np.concatenate((p1, p4), axis=0), np.concatenate((p2, p3), axis=0)])
      return fake_lines_sets

  def parse_color(strcolor):
      strcolor = strcolor[4:-1].split(',')
      strcolor.reverse()
      return tuple(map(int,strcolor))

  #from utils import merge_lines, lines_clustering, find_edge, find_intersection, parse_color, filter_set, check_white
  #import colorbrewer
  folder=""
  _h, _w, _ = img.shape
  e_m_lines = []
  for x1,y1,x2,y2 in m_lines:
      (e_x1,e_y1), (e_x2,e_y2) = find_edge(np.array([x1,y1]), np.array([x2,y2]), _h, _w)
      e_m_lines.append(np.array([e_x1,e_y1,e_x2,e_y2]))
  e_m_lines.sort(key=lambda ele: ele[1])
  idx_non_0 = next((index for index,value in enumerate(np.array(e_m_lines)[:,1].tolist()) if value != 0), None)
  idx_is__h = next((index for index,value in enumerate(np.array(e_m_lines)[:,1].tolist()) if value == _h-1), None)
  e_m_lines[:idx_non_0] = sorted(e_m_lines[:idx_non_0], key=lambda ele: -ele[0])
  if idx_is__h != None:
      e_m_lines[idx_is__h:] = sorted(e_m_lines[idx_is__h:], key=lambda ele: ele[0])

  e_m_lines = merge_lines(e_m_lines)
  # only keep the set that has more than one line
  line_set = [i for i in lines_clustering(e_m_lines, threshold=8) if len(i)>1 and check_white(hsv, e_m_lines, i)]
  print(len(line_set))
  if len(line_set) <=4 :
    continue
  tmp_2 = np.zeros_like(img) + mask.astype('uint8')*50

  fake_sets = filter_set(e_m_lines, line_set, _h, _w)
  tmp_2_f = np.zeros_like(img) + mask.astype('uint8')*50

  stand = [sublist[0] for sublist in fake_sets]

  pp.pprint(stand)
  inter = np.zeros((len(stand),len(stand),2), 'int')
  for i,l1 in enumerate(stand):
      for j,l2 in enumerate(stand):
          if i>=j: continue
          inter[i,j] = np.clip(np.array(find_intersection(l1,l2)), -2000000000, 2000000000)
  np.set_printoptions(suppress=True)
  # pp.pprint(inter[...,0])
  # pp.pprint(inter[...,1])
  intergroup = list()
  intergroup.append({(0,1)})
  for i in range(len(stand)):
      for j in range(len(stand)):
          if i>=j: continue
          new_group = True
          for si,seti in enumerate(intergroup):
              same_group = True
              for ele in seti:
                  if np.linalg.norm(inter[i,j] - inter[ele]) > 100: #TODO Looserfor4avg rulebasefor5avgand6avg
                      same_group = False
                      break
              if same_group:
                  intergroup[si].add((i,j))
                  new_group = False
                  break
          if new_group:
              intergroup.append({(i,j)})
  intergroup.sort(key = lambda ele: -len(ele))
  SET1 = set([ggg for gg in intergroup[0] for ggg in gg])
  SET2 = set([ggg for gg in intergroup[1] for ggg in gg])
  print('SET:', SET1)
  print('SET:', SET2)

  import itertools
  choose1 = list(itertools.combinations(SET1,len(SET1))) + list(itertools.combinations(SET1,len(SET1)))
  choose2 = list(itertools.combinations(SET2,len(SET2))) + list(itertools.combinations(SET2,len(SET2)))
  pp.pprint(choose2)

  print(set([(i[0], i[-1]) for i in choose1]))
  print(set([(i[0], i[-1]) for i in choose2]))
  cand = 0
  maxwin = 0
  for l1,l2 in set([(i[0], i[-1]) for i in choose1]):
      for l3,l4 in set([(i[0], i[-1]) for i in choose2]):
          p1 = np.array(find_intersection(stand[l1],stand[l3]))
          p2 = np.array(find_intersection(stand[l2],stand[l3]))
          p3 = np.array(find_intersection(stand[l2],stand[l4]))
          p4 = np.array(find_intersection(stand[l1],stand[l4]))
          pts = np.array([[p1,p2,p3,p4]]).astype('int32')
          test = np.copy(tmp_2_f)
          color = (255,255,255)
          test = cv2.fillPoly(test, pts, color)

          cand += 1
          test = np.zeros((_h,_w), 'uint8')
          test = cv2.fillPoly(test, pts, (1, 1, 1)).astype(bool)
          if np.logical_and(mask[...,0], test).sum() > maxwin and np.logical_and(mask[...,0], test).sum()*100/test.sum() > 0:
              print('new winner:')
              maxwin = np.logical_and(mask[...,0], test).sum()
              best = ((l1,l2),(l3,l4))
          print('{} {} count {}/{}, {:.2f}, {}'.format((l1,l2),
                                                      (l3,l4),
                                                      np.logical_and(mask[...,0], test).sum(),
                                                      test.sum(),
                                                      np.logical_and(mask[...,0], test).sum()*100/test.sum(),
                                                      '' if np.logical_and(mask[...,0], test).sum()*100/test.sum() < 80 else 'good'))


  ((l1,l2),(l3,l4)) = best
  p1 = np.array(find_intersection(fake_sets[l1][0],fake_sets[l3][0]))
  p2 = np.array(find_intersection(fake_sets[l2][1],fake_sets[l3][0]))
  p3 = np.array(find_intersection(fake_sets[l2][1],fake_sets[l4][1]))
  p4 = np.array(find_intersection(fake_sets[l1][0],fake_sets[l4][1]))
  pts = np.array([[p1,p2,p3,p4]]).astype('int32')

  line1 = [pts[0][1][0], pts[0][1][1], pts[0][3][0], pts[0][3][1]]
  line2 = [pts[0][0][0], pts[0][0][1], pts[0][2][0], pts[0][2][1]]
  s = get_line_cross_point(line1, line2)

  with open('/content/drive/MyDrive/part2_test_image/'+'test_court_detect.csv','a+', newline='') as csvfile1:
      h = csv.writer(csvfile1)
      h.writerow([file,s[0],s[1]])

55 Lines
12
[   array([762,   0, 957, 719]),
    array([741,   0, 907, 719]),
    array([643,   0, 641, 719]),
    array([546,   0, 367, 719]),
    array([543,   0, 376, 719]),
    array([529,   0, 324, 719]),
    array([   0,  369, 1279,  369]),
    array([   0,  380, 1279,  380]),
    array([   0,  455, 1279,  455]),
    array([   0,  542, 1279,  542]),
    array([   0,  650, 1279,  650]),
    array([   0,  674, 1279,  674])]
SET: {6, 7, 8, 9, 10, 11}
SET: {3, 4, 5, 6, 7, 8}
[(3, 4, 5, 6, 7, 8), (3, 4, 5, 6, 7, 8)]
{(6, 11)}
{(3, 8)}
new winner:
(6, 11) (3, 8) count 23982/195859, 12.24, 
12 Lines
0


<ipython-input-4-41dd2b3e7d3b>:2: RuntimeWarning: overflow encountered in int_scalars
  a = y0-y1
<ipython-input-4-41dd2b3e7d3b>:3: RuntimeWarning: overflow encountered in int_scalars
  b = x1-x0
<ipython-input-4-41dd2b3e7d3b>:4: RuntimeWarning: overflow encountered in int_scalars
  c = x0*y1-x1*y0
<ipython-input-4-41dd2b3e7d3b>:11: RuntimeWarning: overflow encountered in int_scalars
  D = a0*b1-a1*b0
<ipython-input-4-41dd2b3e7d3b>:14: RuntimeWarning: overflow encountered in int_scalars
  x = (b0*c1-b1*c0)/D
<ipython-input-4-41dd2b3e7d3b>:15: RuntimeWarning: overflow encountered in int_scalars
  y = (a1*c0-a0*c1)/D


44 Lines
11
[   array([ 764,    0, 1027,  719]),
    array([749,   0, 969, 719]),
    array([641,   0, 641, 719]),
    array([527,   0, 317, 719]),
    array([504,   0, 257, 719]),
    array([   0,  343, 1279,  343]),
    array([   0,  356, 1279,  356]),
    array([   0,  431, 1279,  431]),
    array([   0,  521, 1279,  526]),
    array([   0,  643, 1279,  643]),
    array([   0,  670, 1279,  670])]
SET: {5, 6, 7, 9, 10}
SET: {0, 1, 2, 3, 4}
[(0, 1, 2, 3, 4), (0, 1, 2, 3, 4)]
{(5, 10)}
{(0, 4)}
new winner:
(5, 10) (0, 4) count 196236/203524, 96.42, good
17 Lines
2
21 Lines
2
45 Lines
11
[   array([ 720,    0, 1011,  719]),
    array([708,   0, 956, 719]),
    array([643,   0, 643, 719]),
    array([576,   0, 331, 719]),
    array([562,   0, 273, 719]),
    array([   0,  336, 1279,  336]),
    array([   0,  350, 1279,  350]),
    array([   0,  455, 1279,  455]),
    array([   0,  610, 1279,  610]),
    array([   0,  616, 1279,  604]),
    array([   0,  651, 1279,  651])]
SET: {0, 1, 2, 

In [ ]:
data_path = '/content/drive/MyDrive/part2_test_event'

for file in os.listdir(data_path):
  if file.endswith('.csv'):
    filename = os.path.join(data_path,str(file))
    with open(filename, newline='') as csvFile:
      cout = 0
      rows = csv.reader(csvFile, delimiter=',')
      with open('/content/drive/MyDrive/part2_test_hitter/'+ str(file[:-10]) + '_hitter.csv','w', newline='') as csvfile:
        h = csv.writer(csvfile)
        h.writerow(['frame','Hitter','event','LandingX','LandingY'])
        with open('/content/drive/MyDrive/part2_test_image/test_court_detect.csv', newline='') as csvfile1:
          rss = csv.reader(csvfile1, delimiter=',')
          for lines in rss:
            if str(file[:-10]) == lines[0]:
              thre = lines[2]
              break
        for row in rows:
            if cout == 0:
              cout +=1
              continue
            if cout == 1:
              if float(row[3])<float(thre):
                h.writerow([row[0],'B',row[1],row[2],row[3]])
                k = True
              else:
                h.writerow([row[0],'A',row[1],row[2],row[3]])
                k = False
              cout +=1
            else:
              if k == True:
                h.writerow([row[0],'A',row[1],row[2],row[3]])
                k = False
              else:
                h.writerow([row[0],'B',row[1],row[2],row[3]])
                k = True